In [0]:
!ls

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Nagarjun Mounting Drive code


In [0]:
cd 'content/drive/My Drive/DEEPLEARNING_INTERNSGIP_2020/filter_pruning'

[Errno 2] No such file or directory: '/content/drive/My Drive/DEEPLEARNING_INTERNSGIP_2020/filter_pruning'
/content


Akash Mounting Drive Code


In [2]:
cd 'drive/My Drive/pruned_model'

/content/drive/My Drive/pruned_model


In [0]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_samples, silhouette_score
import scipy.cluster.hierarchy as sch

In [0]:
from models import *
model = vgg(dataset = 'cifar10', depth = 16)

Kmeans CLustering (Which does not seem to work)

In [0]:
weight_reshape = []
Cluster_list = []
layer_number = 1
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        out_channels = m.weight.data.shape[0]
        # ************Reshaping***********        
        shape = m.weight.data.shape
        reshaping = m.weight.data.abs().clone()
        reshaping = reshaping.cpu().numpy().reshape(shape[0] , shape[1]*shape[2]*shape[3])
        weight_reshape.append(reshaping)
        clusters = int(shape[0]/2)

        #******************Calculating silhouette score to automation of kmeans*************
        # Source of this code = https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
        max_sil_score = -1
        for n_cluster in (2,clusters):
          test_kmeans = KMeans(n_clusters = n_cluster, init = 'k-means++', max_iter=100, n_init=1)
          cluster_labels = test_kmeans.fit_predict(reshaping)

          sil_score = silhouette_score(reshaping, cluster_labels)
          if sil_score>max_sil_score:
            max_sil_score = sil_score
            best_n_cluster = n_cluster

        print("Best_n_cluster for "+str(layer_number)+" Conv Layer is "+ str(best_n_cluster))        

        # ***************Kmeans Clustering**********
        kmeans_clustering = KMeans(
          algorithm = 'auto',
          init = 'k-means++',
          max_iter = 300,
          n_clusters = clusters,
          tol = 0.0001,
          n_init = 10,
          random_state = None,
          precompute_distances= 'auto',
          verbose = 0).fit(reshaping)

        clusters = kmeans_clustering.labels_
        Cluster_list.append(clusters)

        layer_number += 1
        # print(Cluster_list)

Best_n_cluster for 1 Conv Layer is 2
Best_n_cluster for 2 Conv Layer is 2
Best_n_cluster for 3 Conv Layer is 2
Best_n_cluster for 4 Conv Layer is 2
Best_n_cluster for 5 Conv Layer is 2
Best_n_cluster for 6 Conv Layer is 2
Best_n_cluster for 7 Conv Layer is 2
Best_n_cluster for 8 Conv Layer is 2
Best_n_cluster for 9 Conv Layer is 2
Best_n_cluster for 10 Conv Layer is 2
Best_n_cluster for 11 Conv Layer is 2
Best_n_cluster for 12 Conv Layer is 2
Best_n_cluster for 13 Conv Layer is 2


AgglomerativeClustering (Finding n_clusters function)

In [0]:
# Source of this code : https://www.programcreek.com/python/example/103493/sklearn.cluster

def compare_k_AggClustering(k_list, X , conv_layer):
    # to find the best k number of clusters
    # X = X.select_dtypes(['number']).dropna()
    # Run clustering with different k and check the metrics
    silhouette_list = []

    for p in k_list:

        clusterer = AgglomerativeClustering(n_clusters=p, linkage="average",affinity = 'cosine')
        
        clusterer.fit(X)
        # The higher (up to 1) the better
        s = metrics.calinski_harabasz_score(X, clusterer.labels_)

        silhouette_list.append(s)

    # The higher (up to 1) the better
    key = silhouette_list.index(max(silhouette_list))

    k = k_list.__getitem__(key)

    return k 

AgglomerativeClustering

In [0]:
#***********Source of this code = https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

from sklearn.cluster import AgglomerativeClustering

Cluster_list = []
weight_reshape = []
conv_layer = 1 
for m in model.modules():
  if isinstance(m , nn.Conv2d):
    shape = m.weight.data.shape
    
    # ************Reshaping*********** 
    reshaping = m.weight.data.abs().clone()
    reshaping = reshaping.cpu().numpy().reshape(shape[0] , shape[1]*shape[2]*shape[3])
    weight_reshape.append(reshaping)

    #*************Finding the n_clusters using the function in the above cell ***************
    n_clusters = compare_k_AggClustering(range(2,int(shape[0]/2)),reshaping,conv_layer)
    print("n_clusters = ", +n_clusters, "Conv_layer = ", +conv_layer)

    #************** Applying agglomerative Clustering *****************
    agglomerative_clustering = AgglomerativeClustering(n_clusters = n_clusters , affinity='cosine', linkage='average')
    res= agglomerative_clustering.fit(reshaping)
    res=sch.fclusterdata(reshaping,t=0.8,criterion='maxclust',method='ward',metric='euclidean')
    labels = agglomerative_clustering.labels_
    Cluster_list.append(labels)
    conv_layer += 1

n_clusters =  8 Conv_layer =  1
n_clusters =  22 Conv_layer =  2
n_clusters =  36 Conv_layer =  3
n_clusters =  27 Conv_layer =  4
n_clusters =  78 Conv_layer =  5
n_clusters =  75 Conv_layer =  6
n_clusters =  70 Conv_layer =  7
n_clusters =  123 Conv_layer =  8
n_clusters =  141 Conv_layer =  9
n_clusters =  123 Conv_layer =  10
n_clusters =  117 Conv_layer =  11
n_clusters =  2 Conv_layer =  12
n_clusters =  124 Conv_layer =  13


In [0]:
cfg = []
cfg_mask = []

for i in range(len(Cluster_list)):
    lst = Cluster_list[i]+2 
    output = []
    appended = set()
    for i in lst :
        if i in appended :
            output.append(0)
        else :
            output.append(i)
            appended.add(i)
    for j in range(len(output)) :
      if output[j] >= 1 :
        output[j] = 1
    cfg.append(int(sum(output)))
    cfg_mask.append(torch.tensor(output))

# print(cfg)
# print(cfg_mask)

In [0]:
from torchsummary import summary
summary(model.cuda(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]          73,728
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,456
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12

In [0]:
from compute_flops import *

print_model_param_nums(model=model, multiply_adds=True)
print_model_param_flops(model=None, input_res=224, multiply_adds=True)

# print_model_param_nums(model=newmodel, multiply_adds=True)
# print_model_param_flops(model=newmodel, input_res=224, multiply_adds=True)

  + Number of params: 14.99M
  + Number of FLOPs: 1.43016G


1430159208.0

Affinity Propogation

In [0]:
#Source of this code = https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html

from sklearn.cluster import AffinityPropagation

Cluster_list = []
weight_reshape = []
conv_layer = 1 
for m in model.modules():
  if isinstance(m , nn.Conv2d):
    shape = m.weight.data.shape

    #***************** Reshaping ***************
    reshaping = m.weight.data.abs().clone()
    reshaping = reshaping.cpu().numpy().reshape(shape[0] , shape[1]*shape[2]*shape[3])
    weight_reshape.append(reshaping)

    #************** Applying Afiinity Propogation *****************
    clustering = AffinityPropagation().fit(reshaping)
    Cluster_list.append(clustering.labels_)
    conv_layer += 1

In [0]:
cfg = []
cfg_mask = []

for i in range(len(Cluster_list)):
    lst = Cluster_list[i]+2 
    output = []
    appended = set()
    for i in lst :
        if i in appended :
            output.append(0)
        else :
            output.append(i)
            appended.add(i)
    for j in range(len(output)) :
      if output[j] >= 1 :
        output[j] = 1
    cfg.append(int(sum(output)))
    cfg_mask.append(torch.tensor(output))

print(cfg)
print(cfg_mask)

[9, 7, 11, 8, 10, 9, 13, 25, 24, 19, 24, 18, 19]
[tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,


Mean Shift

In [0]:
# Source of this code = https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html

from sklearn.cluster import MeanShift

Cluster_list = []
weight_reshape = []
conv_layer = 1 
for m in model.modules():
  if isinstance(m , nn.Conv2d):
    shape = m.weight.data.shape

    #***************** Reshaping ***************
    reshaping = m.weight.data.abs().clone()
    reshaping = reshaping.cpu().numpy().reshape(shape[0] , shape[1]*shape[2]*shape[3])
    weight_reshape.append(reshaping)

    #************** Applying Afiinity Propogation *****************
    clustering = MeanShift().fit(reshaping)
    Cluster_list.append(clustering.labels_)
    conv_layer += 1

In [0]:
cfg = []
cfg_mask = []

for i in range(len(Cluster_list)):
    lst = Cluster_list[i]+2 
    output = []
    appended = set()
    for i in lst :
        if i in appended :
            output.append(0)
        else :
            output.append(i)
            appended.add(i)
    for j in range(len(output)) :
      if output[j] >= 1 :
        output[j] = 1
    cfg.append(int(sum(output)))
    cfg_mask.append(torch.tensor(output))

print(cfg)
print(cfg_mask)

[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0